In [45]:
import openpyxl
from openpyxl.writer.excel import save_workbook
import numpy as np
import pandas as pd
import quantecon as qe
import math
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.tree import export_graphviz
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
from sklearn.multiclass import OneVsOneClassifier
from pycaret.regression import *
from pycaret.classification import *
from pycaret.clustering import *
import tensorflow as tf
from tensorflow import keras
from keras import layers

In [2]:
def transition_matrix(transitions):
    n = 1+ max(transitions)
    M = [[0]*n for _ in range(n)]
    
    for (i,j) in zip(transitions,transitions[1:]):
        M[i][j] += 1
        
    for row in M:
        s = sum(row)
        if s > 0:
            row[:] = [f/s for f in row]
    return M

In [79]:
matrix = np.array([
    [0.000, 0.000, 0.293, 0.359, 0.348],
    [0.000, 0.000, 0.613, 0.280, 0.107],
    [0.000, 0.000, 0.575, 0.010, 0.415],
    [0.556, 0.444, 0.000, 0.000, 0.000],
    [0.241, 0.759, 0.000, 0.000, 0.000]])
list_m = []
list_l = []

for i in range(0, 100):
    mc = qe.MarkovChain(matrix)
    k = mc.simulate(100, init=0)
    list_l.append(k)
    m = transition_matrix(k)
    list_m.append(m)
    
wb = openpyxl.load_workbook('Dataset.xlsx')  
#ws1 = wb.get_sheet_by_name('Лист1')
#row = 1   
#column = 201
#for ar in list_l:
#    for el in ar:
#        ws1.cell(column=column, row=row).value = el
#        row = row + 1
#    column = column + 1
#    row = 1

ws2 = wb.get_sheet_by_name('Matrix')
row = 7002   
column = 1
for mat in list_m:
    for ar in mat:
        for el in ar:
            ws2.cell(column=column, row=row).value = el
            column = column + 1
        row = row + 1
        column = 1
    row = row + 2 
save_workbook(wb, 'Dataset.xlsx')

<ipython-input-79-b90ca983d709>:28: DeprecationWarning: Call to deprecated function get_sheet_by_name (Use wb[sheetname]).
  ws2 = wb.get_sheet_by_name('Matrix')


True

In [80]:
data = pd.read_excel("Dataset.xlsx", "Matrix")

print(data)

      Unnamed: 0  Unnamed: 1  Unnamed: 2  Unnamed: 3  Unnamed: 4
0       0.166667    0.083333    0.250000    0.333333    0.166667
1       0.173913    0.217391    0.086957    0.217391    0.304348
2       0.384615    0.153846    0.153846    0.076923    0.230769
3       0.350000    0.150000    0.050000    0.250000    0.200000
4       0.157895    0.631579    0.105263    0.052632    0.052632
...          ...         ...         ...         ...         ...
7693    0.000000    0.000000    0.333333    0.444444    0.222222
7694    0.000000    0.000000    0.666667    0.238095    0.095238
7695    0.000000    0.000000    0.575000    0.025000    0.400000
7696    0.444444    0.555556    0.000000    0.000000    0.000000
7697    0.200000    0.800000    0.000000    0.000000    0.000000

[7698 rows x 5 columns]


In [81]:
ar = []
mat = []
st = []
i = 6300
while i < 6997:
    if len(mat) == 5:
        i = i + 2
        ar.append(mat)
        mat = []
    else:
        st.append(float(data['Unnamed: 0'][i]))
        st.append(float(data['Unnamed: 1'][i]))
        st.append(float(data['Unnamed: 2'][i]))
        st.append(float(data['Unnamed: 3'][i]))
        st.append(float(data['Unnamed: 4'][i]))
        mat.append(st)
        i = i + 1
        st = []
print(ar)

[[[0.55, 0.0, 0.0, 0.0, 0.45], [0.9090909090909091, 0.09090909090909091, 0.0, 0.0, 0.0], [0.25, 0.375, 0.375, 0.0, 0.0], [0.25, 0.375, 0.125, 0.25, 0.0], [0.08333333333333333, 0.04166666666666666, 0.375, 0.25, 0.25]], [[0.4102564102564102, 0.0, 0.0, 0.0, 0.5897435897435898], [1.0, 0.0, 0.0, 0.0, 0.0], [0.25, 0.5, 0.25, 0.0, 0.0], [0.2857142857142857, 0.2142857142857143, 0.1428571428571428, 0.3571428571428572, 0.0], [0.2222222222222222, 0.1481481481481481, 0.1481481481481481, 0.3333333333333333, 0.1481481481481481]], [[0.35, 0.0, 0.0, 0.0, 0.65], [1.0, 0.0, 0.0, 0.0, 0.0], [0.5555555555555556, 0.1111111111111111, 0.3333333333333333, 0.0, 0.0], [0.6, 0.0, 0.2, 0.2, 0.0], [0.2, 0.2571428571428571, 0.1428571428571428, 0.1142857142857143, 0.2857142857142857]], [[0.35, 0.0, 0.0, 0.0, 0.65], [0.9090909090909091, 0.09090909090909091, 0.0, 0.0, 0.0], [0.3, 0.5, 0.2, 0.0, 0.0], [0.5714285714285714, 0.1428571428571428, 0.1428571428571428, 0.1428571428571428, 0.0], [0.2580645161290323, 0.161290322

In [82]:
ar_c1 = []
ar_c2 = []
c1 = 0
c2 = 0
for i in range(0, len(ar)):
    for j in range(0, len(ar[i][0])):
        c1 = c1 + ar[i][0][j] * j
    ar_c1.append(c1)

for i in range(0, len(ar)):
    for j in range(0, len(ar[i][0])):
        c2 = c2 + (j - ar_c1[i]) ** 2 * ar[i][0][j]
    ar_c2.append(c2)

In [83]:
ar_z = []
a = []

for i in range(0, len(ar)):
    ones_array = np.eye(len(ar[i]), len(ar[i]), dtype=int)
    for j in range(0, len(ar[i])):
        a.append(ar[i][0])
    z = np.linalg.inv(ones_array - (np.array(ar[i]) - np.array(a)))
    ar_z.append(z)
    a = []
ar_z = list(ar_z)

In [84]:
ar_b = []
l_b = []

for i in range(0, len(ar)):
    for j in range(0, len(ar[i])):
        for el in range(0, len(ar[i][j])):
            b = ar[i][0][j] * (2 * ar_z[i][j][el] - 1 - ar[i][0][j])
        l_b.append(b)
    ar_b.append(l_b)
    l_b = []
    
ar_c3 = []
ar_c4 = []
c3 = 0
c4 = 0

for i in range(0, len(ar_b)):
    for j in range(0, len(ar_b[i])):
        c3 = c3 + ar_b[i][j]
        c4 = c4 + ar_b[i][j] * ar_b[i][j]
    ar_c3.append(c3)
    ar_c4.append(math.sqrt(c4))
    c3 = 0
    c4 = 0

In [85]:
ar_c5 = []
ar_c6 = []
mx = 0
c6 = 0

for i in range(0, len(ar)):
    for j in range(0, len(ar[i])):
        for el in range(0, len(ar[i][j])):
            c6 = c6 + ar[i][j][el] * ar[i][j][el]
            if ar[i][j][el] > mx:
                mx = ar[i][j][el]
    ar_c5.append(len(ar[i]) * mx)
    mx = 0
    ar_c6.append(math.sqrt(c6))
    c6 = 0

In [86]:
ar_c7 = []
ar_c8 = []
c7 = 0
s = 0
c8 = 0

for i in range(0, len(ar)):
    for j in range(0, len(ar[i])):
        for el in range(0, len(ar[i][j])):
            #s = s + j * ar[i][j][el]
            s = s + ar[i][j][el]
        for el in range(0, len(ar[i][j])):
            c7 = c7 + (j - s) * (j - s) * ar[i][j][el]
            #print(f'j = {j}, c7 = {c7}, s = {s}, ar={ar[i][j][el]}')
        s = 0
    ar_c7.append(c7 / len(ar[i]))
    c7 = 0
    
for i in range(0, len(ar)):
    for j in range(0, len(ar[i])):
        for el in range(0, len(ar[i][j])):
            c8 = c8 + (ar[i][j][el] - 1 / len(ar[i])) * (ar[i][j][el] - 1 / len(ar[i]))
    ar_c8.append(c8)
    c8 = 0

In [87]:
ar_c9 = []
ar_c10 = []
c9 = 0
c10 = 0
pj = 0

for i in range(0, len(ar)):
    for j in range(0, len(ar[i])):
        for el in range(0, len(ar[i][j])):
            pj = pj + ar[i][j][el]
        pj = pj / len(ar[i])
        for el in range(0, len(ar[i][j])):
            c9 = c9 + (ar[i][el][j] - pj) * (ar[i][el][j] - pj) 
    ar_c9.append(c9)
    c9 = 0
         
for i in range(0, len(ar)):
    for j in range(0, len(ar[i])):
        for el in range(0, len(ar[i][j])):
            if ar[i][j][el] == 0:
                c10 = c10 + ar[i][0][el] * ar[i][j][el] * math.log2(0.0000000001)
            else:
                c10 = c10 + ar[i][0][el] * ar[i][j][el] * math.log2(ar[i][j][el])
    ar_c10.append(-c10)
    c10 = 0

In [88]:
ar_erg = []

from collections import defaultdict

class Graph:

    def __init__(self,vertices):
        self.V= vertices
        self.graph = defaultdict(list)
        
    def addEdge(self,u,v):
        self.graph[u].append(v)   

    def DFSUtil(self,v,visited):
        visited[v]= True
        for i in self.graph[v]:
            if visited[i]==False:
                self.DFSUtil(i,visited)

    def getTranspose(self):
        g = Graph(self.V)
        for i in self.graph:
            for j in self.graph[i]:
                g.addEdge(j,i)
        return g

    def isSC(self):
        visited =[False]*(self.V)
        self.DFSUtil(0,visited)
        if any(i == False for i in visited):
            return False
        gr = self.getTranspose()
        visited =[False]*(self.V)
        gr.DFSUtil(0,visited)
        if any(i == False for i in visited):
            return False
        return True

for i in range(0, len(ar)):
    g = Graph(len(ar[i]))
    for j in range(0, len(ar[i])):
        for el in range(0, len(ar[i][j])):
            if ar[i][j][el] != 0:
                g.addEdge(j, el)
    if g.isSC():
        ar_erg.append(1)
    else:
        ar_erg.append(0)

In [89]:
wb = openpyxl.load_workbook('Dataset.xlsx')  
ws = wb.get_sheet_by_name('Properties')

ws.cell(column=1,row=1).value = 'Мат.ожидание'
ws.cell(column=2,row=1).value = 'Дисперсия распределения'
ws.cell(column=3,row=1).value = 'Норма вектора'
ws.cell(column=4,row=1).value = 'Евклидова норма вектора'
ws.cell(column=5,row=1).value = 'Уровень отклонения 1'
ws.cell(column=6,row=1).value = 'Уровень отклонения 2'
ws.cell(column=7,row=1).value = 'Средняя дисперсия'
ws.cell(column=8,row=1).value = 'Сумма стандартных отклонений элементов'
ws.cell(column=9,row=1).value = 'Разброс по столбцам'
ws.cell(column=10,row=1).value = 'Энтропия'
ws.cell(column=11,row=1).value = 'Эргодичность'

row = 993     
column = 1
for el in ar_c1:
    ws.cell(column=column, row=row).value = el
    row = row + 1

row = 993      
column = 2
for el in ar_c2:
    ws.cell(column=column, row=row).value = el
    row = row + 1

row = 993   
column = 3
for el in ar_c3:
    ws.cell(column=column, row=row).value = el
    row = row + 1

row = 993     
column = 4
for el in ar_c4:
    ws.cell(column=column, row=row).value = el
    row = row + 1
    
row = 993     
column = 5
for el in ar_c5:
    ws.cell(column=column, row=row).value = el
    row = row + 1
    
row = 993      
column = 6
for el in ar_c6:
    ws.cell(column=column, row=row).value = el
    row = row + 1
    
row = 993
column = 7
for el in ar_c7:
    ws.cell(column=column, row=row).value = el
    row = row + 1
    
row = 993      
column = 8
for el in ar_c8:
    ws.cell(column=column, row=row).value = el
    row = row + 1
    
row = 993       
column = 9
for el in ar_c9:
    ws.cell(column=column, row=row).value = el
    row = row + 1
    
row = 993    
column = 10
for el in ar_c10:
    ws.cell(column=column, row=row).value = el
    row = row + 1
    
row = 993     
column = 11
for el in ar_erg:
    ws.cell(column=column, row=row).value = el
    row = row + 1
      
save_workbook(wb, 'Dataset.xlsx')       

<ipython-input-89-27c6322455be>:2: DeprecationWarning: Call to deprecated function get_sheet_by_name (Use wb[sheetname]).
  ws = wb.get_sheet_by_name('Properties')


True

In [34]:
data = pd.read_excel("Dataset.xlsx", "Properties")
data.head(10)

,Мат.ожидание,Дисперсия распределения,Норма вектора,Евклидова норма вектора,Уровень отклонения 1,Уровень отклонения 2,Средняя дисперсия,Сумма стандартных отклонений элементов,Разброс по столбцам,Энтропия,Эргодичность,Д,Р положит(резк),Бп,Р положит(плав.),ЛП,Н_20,Н_40,H_60,Тв,Тн,Бл
0,2.250000,1.687500,-0.849513,0.565326,3.157895,1.185725,3.0,0.405945,0.476779,2.031813,1,1,0,0,0,0,0,0,0,0,0,0
1,4.125000,8.276042,-0.913431,0.538017,2.187500,1.167400,3.0,0.362824,0.425319,2.089298,1,1,0,0,0,0,0,0,0,0,0,0
2,6.183824,27.229383,-0.813233,0.515638,2.250000,1.096878,3.0,0.203140,0.265640,2.176049,1,1,0,0,0,0,0,0,0,0,0,0
3,8.183824,67.469056,-0.777778,0.516914,2.333333,1.138665,3.0,0.296559,0.359059,2.114977,1,1,0,0,0,0,0,0,0,0,0,0
4,10.219538,136.549891,-0.958104,0.607444,2.000000,1.117149,3.0,0.248021,0.310521,2.141810,1,1,0,0,0,0,0,0,0,0,0,0
5,12.157038,242.547438,-0.838710,0.501284,2.631579,1.125600,3.0,0.266976,0.329476,2.198919,1,1,0,0,0,0,0,0,0,0,0,0
6,13.810884,392.490415,-0.919371,0.595912,2.000000,1.104810,3.0,0.220604,0.283104,2.211088,1,1,0,0,0,0,0,0,0,0,0,0
7,15.544217,585.159820,-0.804848,0.510996,2.391304,1.144647,3.0,0.310216,0.372716,2.076354,1,1,0,0,0,0,0,0,0,0,0,0
8,17.321995,828.584980,-0.933734,0.534677,2.142857,1.124852,3.0,0.265292,0.327792,2.042122,1,1,0,0,0,0,0,0,0,0,0,0
9,19.933106,1130.874148,-0.705948,0.501099,1.944444,1.129793,3.0,0.276433,0.338933,2.179893,1,1,0,0,0,0,0,0,0,0,0,0


In [36]:
X = data.drop(['Бп'], axis = 1)
Y = data['Бп']

In [36]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.3, random_state = 42)

In [11]:
# дерево решений
tree = DecisionTreeClassifier(max_depth = 10)
tree_simple = tree.fit(X_train, Y_train)
pred_proba = tree_simple.predict_proba(X_test)[::,0]
fpr, tpr, _ = metrics.roc_curve(Y_test, pred_proba)
auc = metrics.roc_auc_score(Y_test, pred_proba)
predictions = tree_simple.predict(X_test)
print(classification_report(Y_test, predictions))

              precision    recall  f1-score   support

           0       0.97      0.98      0.98       295
           1       0.80      0.75      0.77        32

    accuracy                           0.96       327
   macro avg       0.89      0.86      0.88       327
weighted avg       0.96      0.96      0.96       327



In [17]:
# рандомный лес
tree = RandomForestClassifier(n_estimators=100)
tree_simple = tree.fit(X_train, Y_train)
pred_proba = tree_simple.predict_proba(X_test)[::,0]
fpr, tpr, _ = metrics.roc_curve(Y_test, pred_proba)
auc = metrics.roc_auc_score(Y_test, pred_proba)
predictions = tree_simple.predict(X_test)
print(classification_report(Y_test, predictions))

              precision    recall  f1-score   support

           0       0.97      1.00      0.99       295
           1       1.00      0.75      0.86        32

    accuracy                           0.98       327
   macro avg       0.99      0.88      0.92       327
weighted avg       0.98      0.98      0.97       327



In [20]:
# Логистическая регрессия
classifier = LogisticRegression(solver='lbfgs',random_state=2)
tree_simple = classifier.fit(X_train, Y_train)
pred_proba = tree_simple.predict_proba(X_test)[::,0]
fpr, tpr, _ = metrics.roc_curve(Y_test, pred_proba)
auc = metrics.roc_auc_score(Y_test, pred_proba)
predictions = tree_simple.predict(X_test)
print(classification_report(Y_test, predictions))

              precision    recall  f1-score   support

           0       0.90      1.00      0.95       295
           1       0.00      0.00      0.00        32

    accuracy                           0.90       327
   macro avg       0.45      0.50      0.47       327
weighted avg       0.81      0.90      0.86       327



C:\Users\Dimon\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [31]:
OneVsRestClassifier(LinearSVC(random_state=0)).fit(X, y).predict(X)

IndexError: too many indices for array

In [35]:
X = data.drop(['Бп', 'Д', 'Р положит(резк)', 'Р положит(плав.)', 'ЛП', 'Н_20', 'Н_40', 'Тв', 'Тн', 'Бл'], axis = 1)
Y = data[['Бп', 'Д', 'Р положит(резк)', 'Р положит(плав.)', 'ЛП', 'Н_20', 'Н_40', 'Тв', 'Тн', 'Бл']]

In [10]:
OneVsRestClassifier(LinearSVC(random_state=0)).fit(X, y).predict(X)

,Бп,Д
0,0,1
1,0,1
2,0,1
3,0,1
4,0,1
...,...,...
1085,0,0
1086,0,0
1087,0,0
1088,0,0


In [4]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.3, random_state = 42)

In [5]:
pred_proba = OneVsRestClassifier(LinearSVC(random_state=0)).fit(X_train, Y_train).predict(X_test)
print(classification_report(Y_test, pred_proba))

C:\Users\Dimon\anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\Dimon\anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\Dimon\anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\Dimon\anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\Dimon\anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations

              precision    recall  f1-score   support

           0       0.06      0.25      0.10        32
           1       0.17      0.18      0.18        33
           2       1.00      0.19      0.32        21
           3       0.12      0.95      0.22        37
           4       0.06      0.41      0.10        29
           5       0.15      0.85      0.26        33
           6       0.20      0.04      0.07        24
           7       0.00      0.00      0.00        23
           8       0.23      0.28      0.25        32
           9       0.00      0.00      0.00        29

   micro avg       0.11      0.35      0.17       293
   macro avg       0.20      0.32      0.15       293
weighted avg       0.18      0.35      0.15       293
 samples avg       0.13      0.31      0.18       293



C:\Users\Dimon\anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\Dimon\anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\Dimon\anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\Dimon\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Dimon\anaconda3\lib\site-packages\sklear

In [9]:
pred_proba = OneVsOneClassifier(LinearSVC(random_state=0)).fit(X_train, Y_train).predict(X_test)
print(classification_report(Y_test, pred_proba))

ValueError: y should be a 1d array, got an array of shape (763, 10) instead.

In [ ]:
# Кластеризация, нейронные сети, коханен.

In [16]:
r1 = setup(data, target = 'Н_20', session_id = 123)

,Description,Value
0,session_id,123
1,Target,Н_20
2,Target Type,Binary
3,Label Encoded,None
4,Original Data,"(1090, 22)"
5,Missing Values,False
6,Numeric Features,9
7,Categorical Features,12
8,Ordinal Features,False
9,High Cardinality Features,False


In [17]:
knn = create_model('knn')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.8961,0.2319,0.0000,0.0000,0.0000,0.0000,0.0000
1,0.8961,0.2681,0.0000,0.0000,0.0000,0.0000,0.0000
2,0.9079,0.2536,0.0000,0.0000,0.0000,0.0000,0.0000
3,0.9079,0.3085,0.0000,0.0000,0.0000,0.0000,0.0000
4,0.9079,0.2681,0.0000,0.0000,0.0000,0.0000,0.0000
5,0.9079,0.2319,0.0000,0.0000,0.0000,0.0000,0.0000
6,0.9079,0.2826,0.0000,0.0000,0.0000,0.0000,0.0000
7,0.9079,0.2681,0.0000,0.0000,0.0000,0.0000,0.0000
8,0.9079,0.2826,0.0000,0.0000,0.0000,0.0000,0.0000
9,0.9079,0.2681,0.0000,0.0000,0.0000,0.0000,0.0000


In [19]:
r2 = setup(data = data)

,Description,Value
0,session_id,7757
1,Original Data,"(1090, 22)"
2,Missing Values,False
3,Numeric Features,9
4,Categorical Features,13
5,Ordinal Features,False
6,High Cardinality Features,False
7,High Cardinality Method,None
8,Transformed Data,"(1090, 35)"
9,CPU Jobs,-1


In [20]:
models()

,Name,Reference
ID,,
kmeans,K-Means Clustering,sklearn.cluster._kmeans.KMeans
ap,Affinity Propagation,sklearn.cluster._affinity_propagation.Affinity...
meanshift,Mean Shift Clustering,sklearn.cluster._mean_shift.MeanShift
sc,Spectral Clustering,sklearn.cluster._spectral.SpectralClustering
hclust,Agglomerative Clustering,sklearn.cluster._agglomerative.AgglomerativeCl...
dbscan,Density-Based Spatial Clustering,sklearn.cluster._dbscan.DBSCAN
optics,OPTICS Clustering,sklearn.cluster._optics.OPTICS
birch,Birch Clustering,sklearn.cluster._birch.Birch
kmodes,K-Modes Clustering,kmodes.kmodes.KModes


In [22]:
kmeans = create_model('kmeans')

,Silhouette,Calinski-Harabasz,Davies-Bouldin,Homogeneity,Rand Index,Completeness
0,0.7238,4099.1136,0.4715,0,0,0


In [23]:
kmodes = create_model('kmodes')

,Silhouette,Calinski-Harabasz,Davies-Bouldin,Homogeneity,Rand Index,Completeness
0,-0.3454,7.2864,20.9476,0,0,0


In [24]:
ap = create_model('ap')

,Silhouette,Calinski-Harabasz,Davies-Bouldin,Homogeneity,Rand Index,Completeness
0,0,0,0,0,0,0


In [25]:
meanshift = create_model('meanshift')

,Silhouette,Calinski-Harabasz,Davies-Bouldin,Homogeneity,Rand Index,Completeness
0,0.724,805.3464,0.4604,0,0,0


In [27]:
hclust = create_model('hclust')

,Silhouette,Calinski-Harabasz,Davies-Bouldin,Homogeneity,Rand Index,Completeness
0,0.7312,3183.5029,0.4328,0,0,0


In [28]:
dbscan = create_model('dbscan')

,Silhouette,Calinski-Harabasz,Davies-Bouldin,Homogeneity,Rand Index,Completeness
0,0,0,0,0,0,0


In [29]:
optics = create_model('optics')

,Silhouette,Calinski-Harabasz,Davies-Bouldin,Homogeneity,Rand Index,Completeness
0,0.3463,5.4381,10.9404,0,0,0


In [30]:
birch = create_model('birch')

,Silhouette,Calinski-Harabasz,Davies-Bouldin,Homogeneity,Rand Index,Completeness
0,0.6955,3104.8526,0.4817,0,0,0


In [41]:
model = keras.Sequential([
layers.Dense(35, activation='relu'),
    layers.Dense(70, activation='relu'),
    layers.Dense(70, activation='relu'),
    layers.Dropout(0.1),
    layers.Dense(105, activation='relu'),
    layers.Dense(70, activation='relu'),
    layers.Dense(35, activation='relu'),
    layers.Dense(12)])

optimizer = tf.optimizers.Adam()
model.compile(loss=tf.keras.losses.MSE, optimizer=optimizer, metrics=['accuracy'])
history = model.fit(X, Y, epochs=1000)

Epoch 1/1000

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

35/35 [==============================] - 0s 2ms/step - loss: 14705971.0000 - accuracy: 0.1835
Epoch 2/1000
35/35 [==============================] - 0s 2ms/step - loss: 474540.7500 - accuracy: 0.0716
Epoch 3/1000
35/35 [==============================] - 0s 2ms/step - loss: 186993.5312 - accuracy: 0.0661
Epoch 4/1000
35/35 [==============================] - 0s 2ms/step - loss: 81309.1719 - accuracy: 0.0459
Epoch 5/1000
35/35 [==============================] - 0s 2ms/step - loss: 46186.1797 - 

35/35 [==============================] - 0s 2ms/step - loss: 0.0861 - accuracy: 0.0000e+00
Epoch 150/1000
35/35 [==============================] - 0s 2ms/step - loss: 0.0862 - accuracy: 0.0000e+00
Epoch 151/1000
35/35 [==============================] - 0s 1ms/step - loss: 0.0861 - accuracy: 0.0000e+00
Epoch 152/1000
35/35 [==============================] - 0s 2ms/step - loss: 0.0861 - accuracy: 0.0000e+00
Epoch 153/1000
35/35 [==============================] - 0s 1ms/step - loss: 0.0860 - accuracy: 0.0000e+00
Epoch 154/1000
35/35 [==============================] - 0s 2ms/step - loss: 0.0862 - accuracy: 0.0000e+00
Epoch 155/1000
35/35 [==============================] - 0s 1ms/step - loss: 1.3308 - accuracy: 9.1743e-04
Epoch 156/1000
35/35 [==============================] - 0s 2ms/step - loss: 0.0864 - accuracy: 0.0000e+00
Epoch 157/1000
35/35 [==============================] - 0s 2ms/step - loss: 0.0860 - accuracy: 0.0000e+00
Epoch 158/1000
35/35 [==============================] - 0s 2m

Epoch 302/1000
35/35 [==============================] - 0s 2ms/step - loss: 0.0804 - accuracy: 0.0000e+00: 0s - loss: 0.0816 - accuracy: 0.0000e+
Epoch 303/1000
35/35 [==============================] - 0s 2ms/step - loss: 0.0803 - accuracy: 0.0000e+00
Epoch 304/1000
35/35 [==============================] - 0s 1ms/step - loss: 0.0803 - accuracy: 0.0000e+00
Epoch 305/1000
35/35 [==============================] - 0s 1ms/step - loss: 0.0803 - accuracy: 0.0000e+00
Epoch 306/1000
35/35 [==============================] - 0s 2ms/step - loss: 0.0803 - accuracy: 0.0000e+00
Epoch 307/1000
35/35 [==============================] - 0s 1ms/step - loss: 0.0803 - accuracy: 9.1743e-04
Epoch 308/1000
35/35 [==============================] - 0s 1ms/step - loss: 0.0803 - accuracy: 0.0000e+00
Epoch 309/1000
35/35 [==============================] - 0s 1ms/step - loss: 0.0802 - accuracy: 0.0000e+00
Epoch 310/1000
35/35 [==============================] - 0s 1ms/step - loss: 0.0802 - accuracy: 0.0000e+00
Epoch 

35/35 [==============================] - 0s 1ms/step - loss: 0.0809 - accuracy: 0.0826
Epoch 459/1000
35/35 [==============================] - 0s 1ms/step - loss: 0.0810 - accuracy: 0.0826
Epoch 460/1000
35/35 [==============================] - 0s 1ms/step - loss: 0.0809 - accuracy: 0.0826
Epoch 461/1000
35/35 [==============================] - 0s 2ms/step - loss: 0.0811 - accuracy: 0.0826
Epoch 462/1000
35/35 [==============================] - 0s 1ms/step - loss: 0.0809 - accuracy: 0.0826
Epoch 463/1000
35/35 [==============================] - 0s 1ms/step - loss: 0.0809 - accuracy: 0.0826
Epoch 464/1000
35/35 [==============================] - 0s 1ms/step - loss: 0.0809 - accuracy: 0.0826
Epoch 465/1000
35/35 [==============================] - 0s 1ms/step - loss: 0.0808 - accuracy: 0.0826
Epoch 466/1000
35/35 [==============================] - 0s 1ms/step - loss: 0.0808 - accuracy: 0.0826
Epoch 467/1000
35/35 [==============================] - 0s 2ms/step - loss: 0.0807 - accuracy: 0.

35/35 [==============================] - 0s 2ms/step - loss: 0.0799 - accuracy: 0.0817
Epoch 618/1000
35/35 [==============================] - 0s 2ms/step - loss: 0.0799 - accuracy: 0.0817
Epoch 619/1000
35/35 [==============================] - 0s 1ms/step - loss: 0.0799 - accuracy: 0.0055
Epoch 620/1000
35/35 [==============================] - 0s 2ms/step - loss: 0.0800 - accuracy: 0.0101
Epoch 621/1000
35/35 [==============================] - 0s 2ms/step - loss: 0.0799 - accuracy: 0.0817
Epoch 622/1000
35/35 [==============================] - 0s 2ms/step - loss: 0.0799 - accuracy: 0.0817
Epoch 623/1000
35/35 [==============================] - 0s 2ms/step - loss: 0.0799 - accuracy: 0.0817
Epoch 624/1000
35/35 [==============================] - 0s 2ms/step - loss: 0.0799 - accuracy: 0.0817
Epoch 625/1000
35/35 [==============================] - 0s 2ms/step - loss: 0.0799 - accuracy: 0.0817
Epoch 626/1000
35/35 [==============================] - 0s 2ms/step - loss: 0.0799 - accuracy: 0.

35/35 [==============================] - 0s 2ms/step - loss: 0.0787 - accuracy: 0.0000e+00
Epoch 773/1000
35/35 [==============================] - 0s 2ms/step - loss: 0.0789 - accuracy: 0.0000e+00
Epoch 774/1000
35/35 [==============================] - 0s 1ms/step - loss: 0.0787 - accuracy: 0.0000e+00
Epoch 775/1000
35/35 [==============================] - 0s 2ms/step - loss: 0.0786 - accuracy: 0.0000e+00
Epoch 776/1000
35/35 [==============================] - 0s 1ms/step - loss: 0.0787 - accuracy: 0.0000e+00
Epoch 777/1000
35/35 [==============================] - 0s 1ms/step - loss: 0.0786 - accuracy: 0.0000e+00
Epoch 778/1000
35/35 [==============================] - 0s 1ms/step - loss: 0.0789 - accuracy: 0.0000e+00
Epoch 779/1000
35/35 [==============================] - 0s 2ms/step - loss: 0.0786 - accuracy: 0.0000e+00
Epoch 780/1000
35/35 [==============================] - 0s 1ms/step - loss: 0.0788 - accuracy: 0.0000e+00
Epoch 781/1000
35/35 [==============================] - 0s 2m

35/35 [==============================] - 0s 2ms/step - loss: 0.0721 - accuracy: 0.0000e+00
Epoch 927/1000
35/35 [==============================] - 0s 2ms/step - loss: 0.0716 - accuracy: 0.0000e+00
Epoch 928/1000
35/35 [==============================] - 0s 2ms/step - loss: 0.0729 - accuracy: 0.0000e+00
Epoch 929/1000
35/35 [==============================] - 0s 2ms/step - loss: 0.0726 - accuracy: 0.0000e+00
Epoch 930/1000
35/35 [==============================] - 0s 2ms/step - loss: 0.0713 - accuracy: 0.0000e+00
Epoch 931/1000
35/35 [==============================] - 0s 2ms/step - loss: 0.0724 - accuracy: 9.1743e-04
Epoch 932/1000
35/35 [==============================] - 0s 2ms/step - loss: 0.0709 - accuracy: 0.0000e+00
Epoch 933/1000
35/35 [==============================] - 0s 2ms/step - loss: 0.0722 - accuracy: 0.0000e+00
Epoch 934/1000
35/35 [==============================] - 0s 2ms/step - loss: 0.0718 - accuracy: 0.0000e+00
Epoch 935/1000
35/35 [==============================] - 0s 2m

In [43]:
score = model.evaluate(X_test, Y_test)

11/11 [==============================] - 0s 1ms/step - loss: 0.0802 - accuracy: 0.0000e+00


In [44]:
score

[0.08021222054958344, 0.0]

In [37]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [38]:
classifier = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 42)
classifier.fit(X_train, Y_train)

RandomForestClassifier(criterion='entropy', n_estimators=10, random_state=42)

In [49]:
y_pred = classifier.predict(X_test)
y_test = Y_test.to_numpy()
print(classification_report(Y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      0.56      0.72        32
           1       0.85      0.85      0.85        33
           2       1.00      0.90      0.95        21
           3       0.88      0.76      0.81        37
           4       1.00      0.97      0.98        29
           5       0.91      0.97      0.94        33
           6       0.88      0.92      0.90        24
           7       0.93      0.61      0.74        23
           8       0.12      0.09      0.11        32
           9       0.07      0.07      0.07        29

   micro avg       0.75      0.66      0.71       293
   macro avg       0.76      0.67      0.71       293
weighted avg       0.75      0.66      0.70       293
 samples avg       0.59      0.59      0.59       293

